In [1]:
# load package
import numpy as np
import pandas as pd
import pickle
import xlearn as xl
import logging

# set path
import os
default_path = "/Users/francislin/JrML/03_04_What's Cooking/"
os.chdir(default_path)

# skip interaction warning
import warnings; 
warnings.filterwarnings('ignore')
#你可以选择修改 ast_note_iteractively kernel 选项来使得 Jupyter 为每一行的变量或语句执行这个操作，以便你可以立即看到多条语句一起输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# all_data.pickle is made from prepare_X.ipynb
with open('all_data.pickle','rb') as f:
    df= pickle.load(f)

In [3]:
df.head()

,cuisine,id,used_pepper,used_salt,used_oil,used_garlic,used_ground,used_fresh,used_sauce,used_sugar,...,used_hock,used_rose,used_ciabatta,used_ready,used_regular,used_glass,used_safflower,used_jeera,used_Oil,used_oven
0,greek,10259,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,southern_us,25693,1,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,filipino,20130,1,1,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,indian,22213,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,indian,13162,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
Y_list=[]
for a in df.cuisine:
    if a not in Y_list and not pd.isnull(a):
        Y_list.append(a)
Y_list
len(Y_list)

['greek',
 'southern_us',
 'filipino',
 'indian',
 'jamaican',
 'spanish',
 'italian',
 'mexican',
 'chinese',
 'british',
 'thai',
 'vietnamese',
 'cajun_creole',
 'brazilian',
 'french',
 'japanese',
 'irish',
 'korean',
 'moroccan',
 'russian']

20

In [5]:
# make 20 Y in df
for a in Y_list:
    df['%s'% (a)]= df['cuisine'].str.contains('%s'% (a),na=False)*1

In [6]:
df[102:105]

,cuisine,id,used_pepper,used_salt,used_oil,used_garlic,used_ground,used_fresh,used_sauce,used_sugar,...,thai,vietnamese,cajun_creole,brazilian,french,japanese,irish,korean,moroccan,russian
102,italian,28851,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
103,greek,4635,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
104,korean,7782,0,1,1,1,0,1,1,1,...,0,0,0,0,0,0,0,1,0,0


In [7]:
df_Y= df[(['id','cuisine']+Y_list)]
df_Y.head()

df_key= df[['id','cuisine']]
emb_k5_all=df_key # prepare mother table for first time in the following loops

,id,cuisine,greek,southern_us,filipino,indian,jamaican,spanish,italian,mexican,...,thai,vietnamese,cajun_creole,brazilian,french,japanese,irish,korean,moroccan,russian
0,10259,greek,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,25693,southern_us,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,20130,filipino,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,22213,indian,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,13162,indian,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# 區分train test
df=df.drop(Y_list,axis=1)
dataTrain = df[pd.notnull(df['cuisine'])]
dataTest = df[~pd.notnull(df['cuisine'])]
dataTrain.columns

df_Y_Train= df_Y[pd.notnull(df['cuisine'])]

Index(['cuisine', 'id', 'used_pepper', 'used_salt', 'used_oil', 'used_garlic',
       'used_ground', 'used_fresh', 'used_sauce', 'used_sugar',
       ...
       'used_hock', 'used_rose', 'used_ciabatta', 'used_ready', 'used_regular',
       'used_glass', 'used_safflower', 'used_jeera', 'used_Oil', 'used_oven'],
      dtype='object', length=902)

In [9]:
fm_model = xl.FMModel(task='binary', init=0.1, epoch=10, k=32, lr=0.1, reg_lambda=0.1, opt='sgd', metric='auc')
fm_model

FMModel(alpha=1, beta=1, block_size=500, epoch=10, fold=1, init=0.1, k=32,
    lambda_1=1, lambda_2=1, lr=0.1, metric='auc', model_type='fm',
    n_jobs=4, nthread=None, opt='sgd', reg_lambda=0.1, stop_window=2,
    task='binary')

In [10]:
import logging
def X_embedding_fm(emb_X_input, X_dummies, X_dummies_train, fm_model, Y_col):
    logging.info('Fit the FM model and derive the embedding values')
    fm_model.fit(X_dummies_train, emb_X_input[Y_col]) #Y_col should be a str
    emb_weight = fm_model.weights[1]
    X_emb = np.matmul(X_dummies.values, emb_weight)
    X_emb = pd.DataFrame(X_emb, columns=[Y_col+'_'+str(i) for i in range(emb_weight.shape[1])])
    return X_emb
    

In [11]:
X_dummies = df.drop(['id','cuisine'],axis=1)
X_dummies_train = dataTrain.drop(['id','cuisine'],axis=1)
for y in Y_list:
    emb_X_input = df_Y_Train[['id', y]]#.reset_index(drop=True)
    Y_col=y
    X_emb= X_embedding_fm(emb_X_input, X_dummies, X_dummies_train, fm_model, Y_col)
    emb_k5_all= pd.concat([emb_k5_all,X_emb], axis=1)
    emb_k5_all.shape
emb_k5_all.head()

(49718, 34)

(49718, 66)

(49718, 98)

(49718, 130)

(49718, 162)

...

(49718, 546)

(49718, 578)

(49718, 610)

(49718, 642)

,id,cuisine,greek_0,greek_1,greek_2,greek_3,greek_4,greek_5,greek_6,greek_7,...,russian_22,russian_23,russian_24,russian_25,russian_26,russian_27,russian_28,russian_29,russian_30,russian_31
0,10259,greek,1.356402e-07,1.658690e-07,7.074824e-08,1.529251e-07,5.191751e-08,1.183570e-07,1.032993e-07,7.768111e-08,...,-2.928500e-08,-3.011975e-09,3.551468e-08,-9.313183e-09,1.027598e-08,-3.201952e-09,2.477666e-08,2.400870e-08,1.004919e-08,4.257494e-08
1,25693,southern_us,-3.599593e-08,-3.449493e-08,-4.894840e-08,-3.075692e-08,-3.375822e-08,-3.766603e-08,-4.000164e-08,-4.445520e-08,...,1.859867e-09,8.810141e-08,1.088635e-07,4.218034e-08,5.461292e-08,4.633686e-08,1.138715e-07,8.886790e-08,9.745802e-08,1.208372e-07
2,20130,filipino,1.103896e-05,1.050039e-05,4.419902e-06,5.689063e-06,7.637005e-06,4.067915e-06,1.024807e-05,4.095847e-06,...,3.615708e-06,8.815463e-06,6.749708e-06,1.095223e-05,7.768596e-06,9.094721e-06,2.476099e-06,8.813379e-06,2.013316e-06,9.461340e-06
3,22213,indian,-2.019742e-08,-1.815091e-08,-1.837140e-08,-1.285076e-08,-1.221861e-08,-2.057284e-08,-1.781205e-08,-2.021390e-08,...,1.787777e-08,1.438721e-07,1.233514e-07,7.295086e-08,6.388265e-08,7.278023e-08,1.536631e-07,1.142229e-07,1.402663e-07,1.434954e-07
4,13162,indian,3.954460e-05,1.443133e-05,1.167779e-05,3.141066e-05,2.526310e-05,3.226248e-05,1.061126e-05,3.342805e-05,...,1.498320e-05,2.908074e-05,1.741402e-05,5.240723e-05,5.091304e-05,4.274395e-05,2.789780e-05,2.428468e-05,5.132026e-05,4.562938e-05


In [12]:
# check the global min
emb_k5_all.min().min()

-8.65464978e-07

In [13]:
import pickle
with open('./data/all_data_emb_k32.pickle','wb') as f:
    pickle.dump(emb_k5_all,f)

### embedding one time before loops

In [ ]:
# Just test one time in this cell
emb_X_input = df_Y_Train[['id','greek']]#.reset_index(drop=True)
X_dummies = df.drop(['id','cuisine'],axis=1)
X_dummies_train = dataTrain.drop(['id','cuisine'],axis=1)
Y_col='greek'

In [ ]:
X_emb= X_embedding_fm(emb_X_input, X_dummies, X_dummies_train, fm_model, Y_col)